이 커널은 Just Kaggle 스터디의 일환으로

[bappe](https://www.kaggle.com/bappekim)님의 [Air Pollution in Seoul](https://www.kaggle.com/bappekim/air-pollution-in-seoul) 데이터셋을 살펴보는 커널입니다.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 데이터 불러오기

In [ ]:
df_summary = pd.read_csv('../input/air-pollution-in-seoul/AirPollutionSeoul/Measurement_summary.csv')
df_item = pd.read_csv('../input/air-pollution-in-seoul/AirPollutionSeoul/Original Data/Measurement_item_info.csv')
df_station = pd.read_csv('../input/air-pollution-in-seoul/AirPollutionSeoul/Original Data/Measurement_station_info.csv')

In [ ]:
df_summary.head()

### Measurement date를 date와 time으로 나누고, 일 평균 값으로 합침

In [ ]:
date_time = df_summary['Measurement date'].str.split(" ", n=1, expand=True)
date_time.head()

In [ ]:
df_summary['date'] = date_time[0]
df_summary['time'] = date_time[1]
df_summary = df_summary.drop(['Measurement date'], axis=1)
df_summary.head()

### 일단 서울 전체에 대해서 분석해보기 위해서 date로 groupby하고 분석

In [ ]:
df_seoul = df_summary.groupby(['date'], as_index=False).agg({'SO2':'mean', 'NO2':'mean', 'O3':'mean', 'CO':'mean', 'PM10':'mean', 'PM2.5':'mean'})
df_seoul.head()

In [ ]:
df_seoul.plot(x='date')

In [ ]:
corr = df_seoul.corr()
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

### 미세먼지(PM10) 기준으로 좋음, 보통, 나쁨, 매우나쁨으로 구분

대한민국의 미세먼지 환경기준(일평균)
- 좋음 0~30
- 보통 ~80
- 나쁨 ~150
- 매우나쁨151~

In [ ]:
df_seoul['PM10_class'] = -1
df_seoul.head()

In [ ]:
for (idx, row) in df_seoul.iterrows():
    pm10 = row[5]
    _class = -1
    if pm10 < 0:
        continue
    elif pm10 < 30:
        _class = 0
    elif pm10 < 80:
        _class = 1
    elif pm10 < 150:
        _class = 2
    else:
        _class = 3
    df_seoul.loc[idx, 'PM10_class'] = _class
df_seoul.head()

In [ ]:
df_seoul['PM10_class'].value_counts().plot(kind="bar")

일부로 기준을 그렇게 정했겠지만,

예상과 다르게 보통 > 좋음 > 나쁨 > 매우나쁨 순서로 나타남.

### Examine Strongest Correlation

In [ ]:
sns.jointplot(x=df_seoul["CO"], y=df_seoul["NO2"], kind='kde', xlim=(0,1),ylim=(0,0.13), color='g')
plt.show()